In [1]:
#importando as bibliotecas
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#lendo o arquivo de ações
#Lendo csv

df = pd.read_csv("D:\\acoes\\all_bovespa.csv", delimiter=';')
df

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\acoes\\all_bovespa.csv'

In [ ]:
#Itau
df_itau = df[df['sigla_acao'] == 'ITUB4' ]

In [ ]:
df_itau.tail(10)

In [ ]:
#verificar o tipo do arquivo
df_itau.dtypes

In [ ]:
#Mudar o tipo data
df_itau['data_pregao'] = pd.to_datetime(df_itau['data_pregao'], format='%Y-%m-%d')


In [ ]:
df_itau.dtypes

In [ ]:
df_itau.tail()

In [ ]:
#criando novos campos de medias móveis
df_itau['mm5d'] = df_itau['preco_fechamento'].rolling(5).mean()
df_itau['mm21d'] = df_itau['preco_fechamento'].rolling(21).mean()

In [ ]:
df_itau.head(22)

In [ ]:
#Empurrando para frente os valores das ações
df_itau['preco_fechamento'] = df_itau['preco_fechamento'].shift(-1)

df_itau.head()

In [ ]:
#retirando os dados nulos
df_itau.dropna(inplace=True)
df_itau

In [ ]:
#reindexando o data frame
df_itau = df_itau.reset_index(drop=True)
df_itau

In [ ]:
#verificando quantidade de linhas
qtd_linhas = len(df_itau)

qtd_linhas_treino= round(.70 * qtd_linhas)
qtd_linhas_teste= qtd_linhas - qtd_linhas_treino  
qtd_linhas_validacao = qtd_linhas -1

info = (
    f"linhas treino= 0:{qtd_linhas_treino}"
    f" linhas teste= {qtd_linhas_treino}:{qtd_linhas_treino + qtd_linhas_teste -1}"
    f" linhas validação= {qtd_linhas_validacao}"
)

info


In [ ]:
#separando as features e labels
features = df_itau.drop(['sigla_acao', 'nome_acao', 'data_pregao', 'preco_fechamento'], 1)
labels = df_itau['preco_fechamento']

In [ ]:
#Escolhendo as melhores features com Kbest

features_list = ('preco_abertura','preco_max','preco_minimo','qtd_negocios','volume_negocios','mm5d','mm21d')

k_best_features = SelectKBest(k='all')
k_best_features.fit_transform(features, labels)
k_best_features_scores = k_best_features.scores_
raw_pairs = zip(features_list[1:], k_best_features_scores)
ordered_pairs = list(reversed(sorted(raw_pairs, key=lambda x: x[1])))

k_best_features_final = dict(ordered_pairs[:15])
best_features = k_best_features_final.keys()
print ('')
print ("Melhores features:")
print (k_best_features_final)

In [ ]:
#separando as features escolhidas
features = df_itau.loc[:,['preco_max','preco_minimo','volume_negocios','mm5d']]

In [ ]:
features

In [ ]:
#Separa os dados de treino teste e validação
X_train = features[:qtd_linhas_treino]
X_test = features[qtd_linhas_treino:qtd_linhas_treino + qtd_linhas_teste -1]

y_train = labels[:qtd_linhas_treino]
y_test = labels[qtd_linhas_treino:qtd_linhas_treino + qtd_linhas_teste -1]

print( len(X_train), len(y_train))

print( len(X_test), len(y_test))

In [ ]:
# Normalizando os dados de entrada(features)

# Gerando o novo padrão
scaler = MinMaxScaler()
X_train_scale = scaler.fit_transform(X_train)  # Normalizando os dados de entrada(treinamento)
X_test_scale  = scaler.transform(X_test)       # Normalizando os dados de entrada(teste)



In [ ]:
#treinamento usando regressão linear
lr = linear_model.LinearRegression()
lr.fit(X_train_scale, y_train)
pred= lr.predict(X_test_scale)
cd =r2_score(y_test, pred)

f'Coeficiente de determinação:{cd * 100:.2f}'

In [ ]:
#rede neural
rn = MLPRegressor(max_iter=2000)

rn.fit(X_train_scale, y_train)
pred= rn.predict(X_test_scale)

cd = rn.score(X_test_scale, y_test)


f'Coeficiente de determinação:{cd * 100:.2f}'

In [ ]:
#rede neural com ajuste hyper parameters

rn = MLPRegressor()

parameter_space = {
        'hidden_layer_sizes': [(i,) for i in list(range(1, 21))],
        'activation': ['tanh', 'relu'],
        'solver': ['sgd', 'adam', 'lbfgs'], 
        'alpha': [0.0001, 0.05],
        'learning_rate': ['constant', 'adaptive'],
    }

search = GridSearchCV(rn, parameter_space, n_jobs=-1, cv=5)


search.fit(X_train_scale,y_train)
clf = search.best_estimator_
pred= search.predict(X_test_scale)

cd = search.score(X_test_scale, y_test)

f'Coeficiente de determinação:{cd * 100:.2f}'

In [ ]:
valor_novo = features.tail(1)
valor_novo

In [ ]:
#executando a previsão


previsao=scaler.transform(valor_novo)


pred=lr.predict(previsao)

pred

In [ ]:
df = df[df['sigla_acao'] == 'ITUB4' ]

In [ ]:

data_pregao_full=df['data_pregao']
data_pregao=data_pregao_full.tail(1)

res_full=df['preco_fechamento']
res=res_full.tail(1)

df=pd.DataFrame({'data_pregao':data_pregao, 'real':res, 'previsao':pred})


df.set_index('data_pregao', inplace=True)

print(df)

In [ ]:
# Grafico
plt.figure(figsize=(16,8))
plt.title('Preço das Ações')
plt.plot(df['real'], label='real', color='blue', marker='o')
plt.plot(df['previsao'], label='previsao', color='red', marker='o')
plt.xlabel('data pregao')
plt.ylabel('Preço de Fechamento')
leg = plt.legend()